In [ ]:
import pandas

In [ ]:
import numpy as np

In [ ]:
from pyiron import Project

In [ ]:
from pyiron.atomistics.structure.atoms import Atoms

In [ ]:
from pyiron.lammps.potential import LammpsPotentialFile

In [ ]:
from mendeleev import get_table

In [ ]:
from ase.data import reference_states, atomic_numbers

In [ ]:
ptable = get_table('elements')

In [ ]:
element_lst = ptable.symbol.values.tolist()

In [ ]:
potential_selector = LammpsPotentialFile()

In [ ]:
element_potential_dict = {}
for el in element_lst:
    df_pot = potential_selector.find(el)
    element_potential_dict[el] = np.unique(df_pot[df_pot['Model'] == 'NISTiprpy']['Name'].values).tolist()

In [ ]:
not_working_potential_lst = [
    '2015--Islam-M-M--Li-S--LAMMPS--ipr1',
    '2011--Zhou-X-W--Li-Na-K-Rb-Cs-F-Cl-Br-I--LAMMPS--ipr1',
    '2015--Choudhary-K--Al--LAMMPS--ipr1',
    '2015--Kumar-A--Al-Ni--LAMMPS--ipr1',
    '2015--Kumar-A--Al-Ni-O--LAMMPS--ipr1',
    '2016--Zhou-X-W--Al-Cu--LAMMPS--ipr1',
    '2018--Zhou-X-W--Al-Cu-H--LAMMPS--ipr1',
    '2016--Zhang-P--Ti-O--LAMMPS--ipr1',
    '2016--Zhang-P--Ti-O--LAMMPS--ipr2',
    '2003--Mendelev-M-I--Fe-2--LAMMPS--ipr2',
    '2015--Zhou-X-W--C-Cu--LAMMPS--ipr1',
    '2015--Zhou-X-W--Cu-H--LAMMPS--ipr1',
    '2012--Ward-D-K--Cd-Te-Zn--LAMMPS--ipr1',
    '2013--Ward-D-K--Cd-Te-Zn--LAMMPS--ipr1',
    '2014--Zhou-X-W--Cd-Te-Se--LAMMPS--ipr1',
    '2012--Ward-D-K--Cd-Te--LAMMPS--ipr1',
    '2015--Broqvist-P--Ce-O--LAMMPS--ipr1',
    '2015--Thompson-A-P--Ta--LAMMPS--ipr1',
    # Test
    '2018--Dickel-D-E--Mg-Al-Zn--LAMMPS--ipr1',
    '2015--Pascuet-M-I--Al-U--LAMMPS--ipr1',
    '2009--Kim-H-K--Fe-Ti-C--LAMMPS--ipr1',
    '2015--Ko-W-S--Ni-Ti--LAMMPS--ipr2',
    '2013--Bonny-G--Fe-Cr-W--LAMMPS--ipr1',
    '2013--Bonny-G--Fe-Cr-W-LAMMPS--ipr1',
    '2014--Liyanage-L-S-I--Fe-C--LAMMPS--ipr1',
    '1985--Foiles-S-M--Ni-Cu--LAMMPS--ipr1',
    '1986--Foiles-S-M--Ag-Au-Cu-Ni-Pd-Pt--LAMMPS--ipr1',
    '1989--Adams-J-B--Ag-Au-Cu-Ni-Pd-Pt--LAMMPS--ipr1',
    '2015--Asadi-E--Ni--LAMMPS--ipr1',
]

In [ ]:
element_potential_dict

In [ ]:
pot_lst = []
for k, v in element_potential_dict.items():
    for p in v:
        pot_lst.append(p)

In [ ]:
len(pot_lst), len(np.unique(pot_lst)), len(not_working_potential_lst)

In [ ]:
pr = Project('cohesive')

In [ ]:
pr.remove_jobs(recursive=True)

In [ ]:
def setup_interactive_job(pr, job_name):
    job = pr.create_job(pr.job_type.Lammps, job_name)
    job.server.run_mode.interactive = True
    job.interactive_enforce_structure_reset = True
    return job

In [ ]:
def calc_bulk(potential, element, pr, size=5, job_bulk=None):
    if job_bulk is None:
        close_job = True
        job_bulk = setup_interactive_job(pr=pr, job_name='lmp_bulk') 
        job_bulk.calc_minimize(pressure=0.0)
    else: 
        close_job = False
    structure_bulk = pr.create_ase_bulk(element, cubic=True)
    structure_bulk.set_repeat([size, size, size])
    job_bulk.structure = structure_bulk.copy()
    job_bulk.potential = potential
    job_bulk.run()
    energy_lst = job_bulk.output.energy_tot[-1] / len(structure_bulk)
    volume_lst = job_bulk.output.volume[-1] / len(structure_bulk)
    if close_job:
        job_bulk.interactive_close()
    return energy_lst, volume_lst

In [ ]:
def calc_vac(potential, element, pr, job_vac=None, size=50.):
    if job_vac is None:
        close_job = True
        job_vac = setup_interactive_job(pr=pr, job_name='lmp_vac') 
    else: 
        close_job = False
    structure_vac = Atoms(element, positions=[[0., 0., 0.]], cell=[[size, 0., 0.], [0., size, 0.], [0., 0., size]])
    job_vac.structure = structure_vac.copy()
    job_vac.potential = potential
    job_vac.run()
    energy_lst = job_vac.output.energy_tot.tolist()[-1]
    if close_job:
        job_vac.interactive_close()
    return energy_lst

In [ ]:
job_vac = setup_interactive_job(pr=pr, job_name='lmp_vac') 

In [ ]:
job_bulk = setup_interactive_job(pr=pr, job_name='lmp_bulk') 
job_bulk.calc_minimize(pressure=0.0)

In [ ]:
element_lst = []
potential_lst = []
crystal_structure_lst = []
for k, v in element_potential_dict.items(): 
    ref = reference_states[atomic_numbers[k]]
    if ref is not None and ref['symmetry'] in ['bcc', 'hcp', 'fcc'] and len(v) != 0:
        for p in v:
            if p not in not_working_potential_lst:
                element_lst.append(k)
                potential_lst.append(p)
                crystal_structure_lst.append(ref['symmetry'])

In [ ]:
vac_energy_lst = [
    calc_vac(
        potential=p, 
        element=e, 
        pr=pr, 
        job_vac=job_vac
    )
    for e, p, struct in zip(
        element_lst, 
        potential_lst, 
        crystal_structure_lst
    )
]

In [ ]:
bulk_lst = [
    calc_bulk(
            potential=p, 
            element=e, 
            pr=pr, 
            size=5,
            job_bulk=job_bulk,
        )
    for e, p, struct in zip(
        element_lst, 
        potential_lst, 
        crystal_structure_lst
    )
]

In [ ]:
bulk_energy_lst, volume_lst = zip(*bulk_lst)

In [ ]:
job_bulk.interactive_close()
job_vac.interactive_close()

In [ ]:
df = pandas.DataFrame({'element': element_lst, 'crystal_structure': crystal_structure_lst, 'potential': potential_lst, 'bulk': bulk_energy_lst, 'vac': vac_energy_lst, 'volume': volume_lst})
df

In [ ]:
df.to_csv('cohesive_energy.csv')

In [ ]:
df.to_html('cohesive_energy.html')